# Machine Learning 27: Decission Tree Classification

## **What Decision Tree Classification?**

A **Decision Tree** is a supervised machine learning algorithm used for **classification** (and regression). It works by learning decision rules from data features to predict the class label of instances.

#### 1. **Structure of a Decision Tree:**

* **Root Node:** The first decision point based on the most important feature.
* **Internal Nodes:** Nodes that split based on features.
* **Leaf Nodes (Terminal):** Final decision or class label.

```
Example:
                [Weather?]
               /         \
           Sunny         Rainy
           /                \
       [Humidity?]         Play=No
        /     \
    High      Normal
   /             \
Play=No        Play=Yes
```

### 2. How it Works:

The tree splits the dataset based on **feature values**. At each node, it chooses the **best feature** to split the data to make the **classes as pure as possible** (i.e., mostly one class in each subset).
### 📊 Splitting Criteria (How the best feature is selected):

1. **Gini Impurity:**

   $$
   Gini = 1 - \sum_{i=1}^{n} p_i^2
   $$

   Measures how often a randomly chosen element would be incorrectly labeled.

2. **Entropy & Information Gain:**

   $$
   Entropy = - \sum p_i \log_2(p_i)
   $$

   $$
   Information\ Gain = Entropy(parent) - \sum \frac{N_{child}}{N_{total}} \times Entropy(child)
   $$

3. **Gain Ratio (used in C4.5):**
   Adjusts Information Gain to reduce bias toward features with many values.

### 3. Recursive Partitioning

The tree is built **recursively**:

* Start at root
* Pick the best feature to split using one of the criteria
* Repeat on the resulting subsets
* Stop when:

  * All data in a node are of the same class
  * Maximum depth is reached
  * Minimum samples per leaf is reached

### 4. Pruning (Prevent Overfitting)

Pruning reduces tree size by removing branches that have little power:

* **Pre-pruning:** Stop tree growth early
* **Post-pruning:** Remove branches after the full tree is built

### 5. Advantages

* Easy to understand and interpret
* No need for feature scaling
* Works well with both categorical and numerical data

### 6. Disadvantages

* Can **overfit** on noisy data
* Unstable (small changes in data can lead to a completely different tree)
* Biased with imbalanced classes

### 7. Common Algorithms

* **ID3**: Uses Information Gain
* **C4.5**: Uses Gain Ratio, handles missing data, supports pruning
* **CART** (Classification And Regression Trees): Uses Gini for classification, supports binary splits


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
dataset =pd.read_csv("Social_Network_Ads.csv")

In [ ]:
dataset.drop(["User ID", "Gender"], axis=1, inplace=True)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.isnull().sum()

In [ ]:
sns.scatterplot(x="Age", y="EstimatedSalary", data=dataset, hue="Purchased")
plt.show()

### Splitting features and target

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset['Purchased']

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()
sc.fit(X)
S = pd.DataFrame(sc.transform(X)).colums=X.columns

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Create the classifier
model = DecisionTreeClassifier(max_depth=2)

# Train the model
model.fit(X_train, y_train)

### Make predictions on the test set

In [ ]:
y_pred = model.predict(X_test)
y_pred

### Evalute the model's performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

In [ ]:
plot_decision_regions(X.to_numpy(), y.to_numpy(), clf=model)
plt.show

In [ ]:
from sklearn.tree import plot_tree

In [ ]:
plt.figure(figsize=(12, 8))
plot_tree(
    model,
    filled=True,
    rounded=True
)
plt.title("Decision Tree Visualization", fontsize=16)
plt.show()


In [ ]:
for i in range (1,20):
    dt2 = DecisionTreeClassifier(max_depth=i)
    dt2.fit(X_train,y_train)
    print(dt2.score(X_train,y_train),dt2.score(X_test,y_test),i)

### Decision Tree Split Criteria

When building a decision tree, the algorithm needs a way to decide **which attribute (feature) to split on** at each node. This is where **Entropy**, **Gini Impurity**, and **Information Gain** come in.

### 1. **Entropy (H)** – Measures Uncertainty

Entropy is a concept from **information theory**. It measures the amount of **disorder or impurity** in a dataset.

###  Formula:

For a binary classification:

$$
Entropy(S) = -p_+\log_2(p_+) - p_-\log_2(p_-)
$$

Where:

* $p_+$ = proportion of positive examples in set $S$
* $p_-$ = proportion of negative examples in set $S$

### Properties:

* Entropy = 0 → perfect purity (all one class)
* Entropy = 1 → maximum impurity (classes evenly split)

### Example:


If a dataset has 50% positive and 50% negative samples:

$$
Entropy = -0.5 \log_2(0.5) - 0.5 \log_2(0.5) = 1
$$

### 2. **Information Gain (IG)** – Measures Reduction in Uncertainty

Information Gain tells us **how much entropy is reduced** by splitting the dataset on an attribute.

### Formula:

$$
IG(S, A) = Entropy(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} \cdot Entropy(S_v)
$$

Where:

* $S$ is the original dataset
* $A$ is the attribute we're splitting on
* $S_v$ is the subset of $S$ where attribute $A = v$

We **choose the attribute with the highest Information Gain** for the split.

## 3. **Gini Impurity (G)** – Another Measure of Impurity

Gini Impurity measures how often a randomly chosen element from the set would be incorrectly labeled.

### Formula:

For binary classification:

$$
Gini(S) = 1 - p_+^2 - p_-^2
$$

### Properties:

* Gini = 0 → perfect purity
* Gini = 0.5 → max impurity in binary classification

### Comparison with Entropy:

* Gini is often faster to compute than entropy.
* Both give similar results, but **Gini is more sensitive to class imbalance**.
* **CART (Classification and Regression Trees)** uses Gini by default.

### Summary Table

| Measure          | Purpose               | Range     | Best Split Criterion     |
| ---------------- | --------------------- | --------- | ------------------------ |
| Entropy          | Measures impurity     | \[0, 1]   | Highest Information Gain |
| Gini Impurity    | Measures impurity     | \[0, 0.5] | Lowest Gini              |
| Information Gain | Measures entropy drop | \[0, 1]   | Highest IG               |

### ✅ Final Notes:

* Use **Entropy + Information Gain** if you want more theoretical interpretability.
* Use **Gini Impurity** for faster computation (most practical cases).
* In **scikit-learn**, use:

  * `criterion='entropy'` → Entropy + IG
  * `criterion='gini'` → Gini Impurity (default)
